In [1]:
import pandas as pd
import datetime
import numpy as np

file_dir = '/Users/leec813/OneDrive - PNNL/Documents/wfip2-iea36/'+\
    'benchmark-original-submissions/WEPROG/mseps_forecasts/'

In [2]:
def process_weprog_data(file_name):

    df = pd.read_csv(file_dir+file_name, skiprows=8, delimiter='\t', header=0)

    new_df = df.reset_index().T.reset_index().T.reset_index().drop(['index', 0], axis=1)

    # duplicating values in first row leads to 'xx.1' column names
    for ind, col in enumerate(new_df.iloc[0]):
        if len(col.split('.')) > 2:
            new_df.iloc[0][ind+1] = '.'.join(col.split('.')[:2])

    new_df.iloc[0][1:] = new_df.iloc[0][1:].astype(float)

    model_hgt_list = np.sort(np.array([35, 100, 170]))

    new_df.loc[(new_df[2] == 10), 2] = 'ws_35'
    new_df.loc[(new_df[2] == 11), 2] = 'ws_100'
    new_df.loc[(new_df[2] == 12), 2] = 'ws_170'

    new_df.loc[(new_df[2] == 34), 2] = 'wd_100'
    new_df.loc[(new_df[2] == 55), 2] = 'wd_170'
    new_df.loc[(new_df[2] == 56), 2] = 'wd_35'

    for site in new_df[1].unique(): 

        if site == 'WFIP2':
            target_hgt_list = [40, 80, 120]
        elif site == 'FINO2':
            target_hgt_list = [62, 72, 82, 92]
        elif site == 'BALTIC2':
            target_hgt_list = [78.25]

        piv_df = new_df.loc[new_df[1] == site].pivot(index=3, columns=2).drop(1, axis=1)

        date_list = [datetime.datetime.strptime(file_name.split('_')[0], '%Y%m%d%H')
                     +datetime.timedelta(hours=1*x) for x in range(len(piv_df))]

        piv_df.index = date_list

        # first column yield error...
        for member in piv_df.columns.levels[0][1:]:

            ws_df = piv_df[member].filter(like='ws')

            out_df = pd.DataFrame()

            for t_hgt in target_hgt_list:

                # given only 3 heights in model, select 2 indices (of heights) that are closest to t_hgt
                two_hs = np.where(abs(t_hgt-model_hgt_list) != abs(t_hgt-model_hgt_list).max())[0]

                alpha = np.log( (ws_df.filter(like=str(model_hgt_list[two_hs[1]])).squeeze()
                                 /ws_df.filter(like=str(model_hgt_list[two_hs[0]])).squeeze()).astype(float) )\
                    /np.log(model_hgt_list[two_hs[1]]/model_hgt_list[two_hs[0]])

                out_df['wind-speed_'+str(t_hgt)+'m'] = ws_df.filter(like=str(model_hgt_list[two_hs[1]])).squeeze()\
                    *(t_hgt/model_hgt_list[two_hs[1]])**alpha

            out_df.index.name = 'time'

            out_df.to_csv(file_dir+'processed/weprog_'+site+'_m'+str(member-3)+'.csv')

In [3]:
process_weprog_data('2016092312_048h_mseps.csv')

In [4]:
process_weprog_data('2020100400_168h_mseps.csv')